In [94]:
import numpy as np
import plotly.graph_objects as go

The agent has a Constant relative Risk Aversion CRRA utility function.

In [95]:
def utility(c, sigma):
    if sigma!=1:
        return (c**(1 - sigma)) / (1 - sigma)
    else:
        return np.log(c)

We plot the utility function for differents levels of risk aversion.

In [96]:
c = np.linspace(0.1,50,100)
sigmas = np.array([0.1,0.3,0.5,1,2])

fig = go.Figure()

for sigma in sigmas:
    utilities = utility(c,sigma)
    fig.add_trace(go.Scatter(
        x=c,
        y=utilities,
        mode='lines',
        name=f'σ={sigma}'
))

fig.update_layout(
    title="Utility Function (CRRA)",
    xaxis_title="Consumption (c)",
    yaxis_title="Utility U(c)")
fig.show()

The objective is to compute this Bellman Equation: 
$$
V(H_t) = \max_{L_t \in [0,1]} [ U(C_t) + \beta V(H_{t+1})] \\
$$
Subject to:
$$
(i)\quad C_t = H_t^\alpha L_t \\
$$
$$
(ii)\quad H_{t+1} = (1-\delta)H_t + (1 - L_t)
$$

# 1. Approximation of the value function

In [97]:
# Parameters
alpha = 0.4
delta = 0.05
beta = 0.9
sigma = 0.9

# Grid for H
H_min, H_max, H_points = 0.1, 1.0, 200
H_values = np.linspace(H_min, H_max, H_points)

# First guess for the value function: 0 for every value of H
V = np.zeros(H_points)

In [98]:
def bellman_update(V, H_values, alpha, beta, delta, sigma):
    new_V = np.zeros_like(V)
    policy = np.zeros_like(V)  # Store optimal L for each H

    for i, H in enumerate(H_values):
        max_value = -np.inf
        optimal_L = 0

        L_candidates = np.linspace(0, 1, 100)

        for L in L_candidates:
            C = (H**alpha) * L
            H_next = (1 - delta) * H + (1 - L)

            # Prevent storing invalid values
            if H_next < H_min or H_next > H_max or C <= 0:
                continue

            # Current utility + discounted future value
            value = utility(C, sigma) + beta * np.interp(H_next, H_values, V)

            if value > max_value:
                max_value = value
                optimal_L = L

        new_V[i] = max_value
        policy[i] = optimal_L

    return new_V, policy


In [99]:
tolerance = 1e-5
max_iterations = 500

for iteration in range(max_iterations):
    new_V, policy = bellman_update(V, H_values, alpha, beta, delta, sigma)
    # error = np.max(np.abs(new_V - V))
    # print("Iteration "+str(iteration)+'\n Error is '+str(error)+'\n') if iteration % 50 == 0 or error < tolerance else None
    if np.max(np.abs(new_V - V)) < tolerance:
        print(f"Converged in {iteration} iterations")
        break
    V = new_V

Iteration 0
 Error is 10.000000000000002

Iteration 50
 Error is 0.051201643627337035

Iteration 100
 Error is 0.00026388176152636333

Iteration 132
 Error is 9.060865366450344e-06

Converged in 132 iterations


# 2. Graphics

## 2.1 Value function

In [100]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=H_values,
    y=V,
    mode = "lines",
    name = "approximated value function"
))
fig.update_layout(
    xaxis_title="Human Capital (H)",
    yaxis_title="Value function",
    title={
    "text": "Approximation of the Value Function<br><sup>Parameters: α = 0.4, β = 0.9, σ = 0.9, δ = 0.05</sup>",
    "x": 0.5,  # Center align title
    "xanchor": "center"
    },
    # showlegend=True
)
fig.show()

## 2.2 Policy function

$$
(ii)\quad H_{t+1} = (1-\delta)H_t + (1 - L_t)
$$

In [101]:
Hnext = (1 - delta) * H_values + (1-policy)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=H_values,
    y=Hnext,
    mode = "lines",
    name='Optimal Policy'
))
fig.add_trace(go.Scatter(
    x=H_values,
    y=H_values,
    mode="lines",
    name="45° line"
))
fig.update_layout(
    title = {
        "text":"Optimal Policy for Human Capital accumulation",
        "x": 0.5,
        "xanchor": "center"
    },
    xaxis_title='Ht',
    yaxis_title='Ht+1',
    showlegend=True
)
fig.show()

## 2.3 Consumption and Labour Supply according to a  stock of Human Capital

In [130]:
consumption = (H_values**alpha)*policy

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=H_values,
    y=policy,
    mode = "lines",
    name='optimal L',
))
fig.add_trace(go.Scatter(
    x=H_values,
    y=consumption,
    name="optimal C"
))
fig.update_layout(
    title = {
        "text":"Consumption and Labour Supply according to the stock of human capital",
        "x": 0.5,
        "xanchor": "center"
    },
    xaxis_title='Human Capital (H)',
    # yaxis_title='Labour supply (L)',
    showlegend=True
)
fig.show()

# 3. Compute a Value function for any arbitrary function

# 3.1 Define an arbitrary policy function